In [1]:
# !pip install textblob

# !pip install tweepy

# !pip install pycountry

# !pip install wordcloud 

# !pip install langdetect

In [19]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/farrisatif/nltk_data...


True

In [2]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from datetime import timedelta
import datetime
import pytz

In [4]:
# import config
# # put ur keys in a py file as a dictionary and import them so u can keep em secret
# consumer_key = config.params['consumer_key'] #api key 
# consumer_secret = config.params['consumer_secret'] #api secret 
# access_token = config.params['access_token'] #access token  
# access_token_secret = config.params['access_token_secret'] #access secret  
# bear = config.params['bear'] #bearer token

In [6]:
# refer to repo .README for necessary format for keys.py

from keys import *

In [7]:
client = tweepy.Client(bearer_token=bearer_token,\
                       consumer_key = consumer_key,\
                       consumer_secret = consumer_secret,\
                       access_token = access_token,\
                       access_token_secret = access_token_secret)

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [9]:
# tweets = tweepy.Cursor(api.search_tweets, 'warriors',
#                        lang="en", start_date=game_datetime_start, until=game_datetime_end).items(10)

# print ([tweet for tweet in tweets])

In [13]:
# pulling NBA game specific data

df_data = pd.read_csv("finalNbaData.csv").drop('Unnamed: 0', axis=1)
# df_odds = pd.read_csv("playoffOdds.csv")
# df_results = pd.read_csv("playoffResults.csv")

df_data.dateTime = pd.to_datetime(df_data.dateTime)

utc=pytz.UTC
weekPriorDate = datetime.datetime.now() - timedelta(days=6)
weekPriorDate = utc.localize(weekPriorDate) 

df_data.dateTime = df_data.dateTime.apply(lambda x: x.replace(tzinfo=None))

# pulling games that occur within 1 week of today's date

study_games = df_data[df_data.dateTime > weekPriorDate.replace(tzinfo=None)].__id__.unique()

In [14]:
study_games

array(['BOSMIL512022', 'BOSMIL532022', 'MEMGS512022', 'MEMGS532022',
       'MIAPHI522022', 'MIAPHI542022', 'PHODAL522022', 'PHODAL542022'],
      dtype=object)

In [15]:
# for tweet in tweepy.Paginator(client.search_recent_tweets, query='Warriors',
#                               tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=1000):
#     print(tweet.id)

In [16]:
# tweets = client.search_recent_tweets(query=home_team, max_results=100, start_time=game_datetime_start, end_time=game_datetime_end, tweet_fields='lang')

In [17]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

def clean_tweets(tweet_list):
    tweet_list = [re.sub('RT @\w+: '," ",x) for x in tweet_list]
    tweet_list = [re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x) for x in tweet_list]
    tweet_list = [x.lower() for x in tweet_list]
    return tweet_list

In [20]:
sentiment_results = pd.DataFrame(columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed'])
search_limit = 500

for game_id in study_games:
    df = df_data[df_data.__id__ == game_id]
    home_team = df['homeTeam'].values[0]
    away_team = df['awayTeam'].values[0]

    game_datetime = pd.to_datetime(df['dateTime'].values[0])
    game_datetime_end = game_datetime
    game_datetime_start = game_datetime - timedelta(hours=5, minutes=0)
    
    
    print (game_id, home_team, away_team)
    
    # home tweets
    home_tweet_list = []
    home_negative = 0
    home_positive = 0
    home_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=home_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            home_tweet_list.append(tweet.text)
     
    home_tweet_count = len(set(home_tweet_list))
    
    for tweet in clean_tweets(list(set(home_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            home_negative += 1
        elif pos > neg:
            home_positive += 1
        elif pos == neg:
            home_neutral += 1
    
    # away tweets
    away_tweet_list = []
    away_negative = 0
    away_positive = 0
    away_neutral = 0
    
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=away_team,
                              tweet_fields=['lang', 'created_at'], max_results=100, start_time=game_datetime_start, end_time=game_datetime_end).flatten(limit=search_limit):

        if tweet.lang == 'en':
            away_tweet_list.append(tweet.text)
    
    away_tweet_count = len(set(away_tweet_list))
    
    for tweet in clean_tweets(list(set(away_tweet_list))):
        score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']

        if neg > pos:
            away_negative += 1
        elif pos > neg:
            away_positive += 1
        elif pos == neg:
            away_neutral += 1
    
    sentiment_results = sentiment_results.append(pd.DataFrame([[game_id, home_team, away_team, percentage(home_positive, home_tweet_count), percentage(home_negative, home_tweet_count), percentage(home_neutral, home_tweet_count),
                                        percentage(away_positive, away_tweet_count), percentage(away_negative, away_tweet_count), percentage(away_neutral, away_tweet_count), away_tweet_count + home_tweet_count]], 
                                           columns=['gameID', 'homeTeam', 'awayTeam','homePositive', 'homeNegative', 'homeNeutral', 'awayPositive', 'awayNegative', 'awayNeutral', 'total_tweets_analyzed']))


BOSMIL512022 Boston Celtics Milwaukee Bucks
BOSMIL532022 Boston Celtics Milwaukee Bucks
MEMGS512022 Memphis Grizzlies Golden State Warriors
MEMGS532022 Memphis Grizzlies Golden State Warriors
MIAPHI522022 Miami Heat Philadelphia 76ers
MIAPHI542022 Miami Heat Philadelphia 76ers
PHODAL522022 Phoenix Suns Dallas Mavericks
PHODAL542022 Phoenix Suns Dallas Mavericks


In [21]:
sentiment_results

,gameID,homeTeam,awayTeam,homePositive,homeNegative,homeNeutral,awayPositive,awayNegative,awayNeutral,total_tweets_analyzed
0,BOSMIL512022,Boston Celtics,Milwaukee Bucks,38.565022,6.278027,55.156951,37.198068,2.898551,59.903382,430
0,BOSMIL532022,Boston Celtics,Milwaukee Bucks,54.166667,5.833333,40.000000,55.421687,3.212851,41.365462,489
0,MEMGS512022,Memphis Grizzlies,Golden State Warriors,48.214286,3.571429,48.214286,53.080569,1.895735,45.023697,435
0,MEMGS532022,Memphis Grizzlies,Golden State Warriors,37.142857,10.952381,51.904762,46.031746,8.730159,45.238095,462
0,MIAPHI522022,Miami Heat,Philadelphia 76ers,54.393305,7.949791,37.656904,50.490196,6.862745,42.647059,443
0,MIAPHI542022,Miami Heat,Philadelphia 76ers,39.700375,13.483146,46.816479,31.981982,14.864865,53.153153,489
0,PHODAL522022,Phoenix Suns,Dallas Mavericks,42.748092,11.068702,46.183206,56.569343,4.744526,38.686131,536
0,PHODAL542022,Phoenix Suns,Dallas Mavericks,49.765258,9.859155,40.375587,47.027027,7.567568,45.405405,398
